In [1]:
import requests
from bs4 import BeautifulSoup
from typing import List, Dict
import pandas as pd



In [3]:
page = requests.get("https://www.ikea.lt/lt/search/?q=sofa")
#page.status_code
#page.content
soup = BeautifulSoup(page.content, 'html.parser')

print(soup.prettify())

all_h1 = soup.find_all("h1")
all_p = soup.find_all("p")
all_divs = soup.find_all("div")


<!DOCTYPE html>
<html lang="lt">
 <head>
  <script async="" data-cbid="118aba1e-8764-430e-ad27-bbc95761e72a" data-consentmode="disabled" data-culture="lt" id="Cookiebot" src="https://consent.cookiebot.com/uc.js" type="text/javascript">
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   IKEA Lietuva
  </title>
  <meta content="IKEA Lietuva" name="title"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no, shrink-to-fit=no" name="viewport"/>
  <style>
   @charset "UTF-8";@import url(https://fonts.googleapis.com/css?family=Noto+Sans:400,400i,700,700i&display=swap&subset=cyrillic,cyrillic-ext,latin-ext);@font-face{font-family:'VerdanaIKEA';src:url(https://d28of4jgobvr79.cloudfront.net/webroot/fonts/VerdanaIKEA/VerdanaIKEA_regular.ttf) format("truetype");font-weight:normal;font-style:normal}@font-face{font-family:'VerdanaIKEA';src:url(https://d28of4jgobvr79.cloudfront.net/webroot/fonts/Verdana

In [21]:
#type(all_divs[0])

bs4.element.Tag

In [11]:
print(type(all_p[0]))
print(all_p[0].text)

<class 'bs4.element.Tag'>
Išbandykite receptą iš knygos „Desertai. 365 receptai“.


In [5]:
all_recipe_divs = soup.find_all("div", class_="list-row")

results: List[Dict] = []

for recipe_div in all_recipe_divs:
    recipe_title = recipe_div.find("h3").text.strip().split('\n')[0] #test paima teksta, strip pasalina \n, split paima pirma eilut stringo
    recipe_amount = recipe_div.find("span").text # keliems zmonems
    recipe_link = recipe_div.find("a").get("href")
    recipe_image_link = recipe_div.find("img").get("src")
    #print(recipe_title)
    #print(recipe_link)
    #print(recipe_amount)
    #print(recipe_image_link)

    results.append({
        "title": recipe_title,
        "recipe_amount": recipe_amount,
        "recipe_link": recipe_link,
        "recipe_image_link": recipe_image_link

    })

#results

#results_data_frame = pd.DataFrame(results)
#results_data_frame.to_csv("15min.csv")

In [18]:
## RECEPTU SARASAS

def extract_recipes_info(url: str) -> List[Dict]:
    results: List[Dict] = []
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception("Cannot reach page.")
    
    soup = BeautifulSoup(resp.content, 'html.parser')

    # aprasome html koses elementa is kurios surinksim info
    all_recipe_divs = soup.find_all("div", class_="list-row")
    
    for recipe_div in all_recipe_divs:
        recipe_title = recipe_div.find("h3").text.strip().split('\n')[0] #text paima teksta, strip pasalina \n, split paima pirma eilut stringo
        #recipe_amount = recipe_div.find("span").text
        recipe_link = recipe_div.find("a").get("href")
        recipe_image_link = recipe_div.find("img").get("src")

        results.append({
            "title": recipe_title,
            #"recipe_amount": recipe_amount,
            "recipe_link": recipe_link,
            "recipe_image_link": recipe_image_link

        })

    return results



In [20]:
extract_recipes_info("https://www.15min.lt/maistas/receptai?psl=2000")
#type(test)
#ilgis = len(test)
#ilgis

resp yra <Response [200]>


[]

In [6]:
result: List[Dict] = []

for page in range(20, 80):
    url = f"https://www.15min.lt/maistas/receptai?psl={page}"
    result.extend(extract_recipes_info(url))

pd.DataFrame(result).to_csv("15min.csv")

In [46]:
## Ingredientai
resp = requests.get("https://www.15min.lt/maistas/receptas/varskes-ir-fermentinio-surio-pyragas-8796")
soup = BeautifulSoup(resp.content, "html.parser")
all_ingredients: List[Dict] = []

recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
recipe_image = soup.find("div", class_="image").find("img").get("src")
recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
for ingredient in recipe_ingredients:
    the_ingredient = ingredient.find_all("span")
    all_ingredients.append({
        "ingredient_name": the_ingredient[0].text.strip(),
        "ingredient_amount": the_ingredient[2].text.strip(),
        "ingredient_note": the_ingredient[3].text.strip()
    })

#all_ingredients

## Paruosimo budas
recipe_manual: List[str] = []
recipe_making_steps = soup.find("div", class_="description text").find_all("p")

recipe_making_steps



for step in recipe_making_steps:
    stepsas = step.text.strip()
    recipe_manual.append(stepsas)

In [35]:
## Info apie viena recepta

def extract_recipe_ingredients(url: str) -> str:
    ## Ingredientai
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    all_ingredients: List[Dict] = []

    recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
    recipe_image = soup.find("div", class_="image").find("img").get("src")
    recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
    for ingredient in recipe_ingredients:
        ingredient_span = ingredient.find_all("span")
        all_ingredients.append({            
                "ingredient_name": ingredient_span[0].text.strip(),
                "ingredient_amount": ingredient_span[2].text.strip(),
                "ingredient_note": ingredient_span[3].text.strip()
            })
       
    return str(all_ingredients)


def extract_making_steps(url: str) -> str:

    ## Paruosimo budas
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    recipe_manual: List[str] = []
    recipe_making_steps = soup.find("div", class_="description text").find_all("p")

    for step in recipe_making_steps:
        step_to_txt = step.text.strip()
        recipe_manual.append(step_to_txt)
    
    return "\n".join(recipe_manual) 


In [36]:
ing = extract_recipe_ingredients("https://www.15min.lt/maistas/receptas/makaronai-su-gorgonzolos-padazu-8788")

#manual = extract_making_steps("https://www.15min.lt/maistas/receptas/makaronai-su-gorgonzolos-padazu-8788")

print(ing)
#print(manual)

[{'ingredient_name': 'Makaronai', 'ingredient_amount': '250 g', 'ingredient_note': ''}, {'ingredient_name': 'Smidrai', 'ingredient_amount': '10 vnt.', 'ingredient_note': ''}, {'ingredient_name': 'Šalotinis svogūnas', 'ingredient_amount': '1 vnt.', 'ingredient_note': ''}, {'ingredient_name': 'Šoninė', 'ingredient_amount': '200 g', 'ingredient_note': 'rūkytos'}, {'ingredient_name': 'Pomidorai', 'ingredient_amount': '1 sauja', 'ingredient_note': 'vyšninių'}, {'ingredient_name': 'Špinatai', 'ingredient_amount': '1 sauja', 'ingredient_note': ''}, {'ingredient_name': 'Gorgonzolos sūris', 'ingredient_amount': '150 g', 'ingredient_note': ''}, {'ingredient_name': 'Grietinėlė', 'ingredient_amount': '70 ml', 'ingredient_note': ''}, {'ingredient_name': 'Parmezano sūris', 'ingredient_amount': '1 valg. š.', 'ingredient_note': 'tarkuoto'}]


In [23]:
page = requests.get("https://www.ikea.lt/lt/search/?q=sofa")
#page.status_code
#page.content
soup = BeautifulSoup(page.content, 'html.parser')

products: List[Dict] = []
search_results_all = soup.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")

for product in search_results_all:    
    products.append({            
                "product_name": product.find("div", class_="itemInfo").a.text.strip(),
                "product_info": product.find("div", class_="itemInfo").find("div", class_ = "itemPrice-wrapper").span.text.replace(" €", ""),
                "product_link": product.find("div", class_="itemInfo").a.get("href"), # prideti https://www.ikea.lt/
                "product_image": product.find("div", class_ = "productImg").img.get("src")
            })

products
    


[{'product_name': 'SÖDERHAMN',
  'product_info': '600',
  'product_link': '/lt/products/ikea-verslui/nekilnojamasis-turtas/sofos/soderhamn-2-viete-sofa-su-gulimuoju-kreslu-naturalios-sp-spr-19442140',
  'product_image': 'https://d2rbyiw1vv51io.cloudfront.net/web/ikea4/images/574/1057490_PE848852_S3.jpg'},
 {'product_name': 'SÖDERHAMN',
  'product_info': '530',
  'product_link': '/lt/products/ikea-verslui/nekilnojamasis-turtas/sofos/soderhamn-2-viete-sofa-su-gulimuoju-kreslu-pilka-spr-29452100',
  'product_image': 'https://d2rbyiw1vv51io.cloudfront.net/web/ikea4/images/578/1057810_PE848992_S3.jpg'},
 {'product_name': 'SÖDERHAMN',
  'product_info': '600',
  'product_link': '/lt/products/ikea-verslui/nekilnojamasis-turtas/sofos/soderhamn-2-viete-sofa-su-gulimuoju-kreslu-tamsiai-pilka-spr-39449616',
  'product_image': 'https://d2rbyiw1vv51io.cloudfront.net/web/ikea4/images/576/1057692_PE848907_S3.jpg'},
 {'product_name': 'VIMLE',
  'product_info': '259',
  'product_link': '/lt/products/ike